# Kitaev chain spectrum with Kwant
$$\newcommand{\ket}[1]{\left|{#1}\right\rangle}
\newcommand{\bra}[1]{\left\langle{#1}\right|}$$
In order to use Kwant to solve the system, we need to write the BdG Hamiltonian of the Kitaev chain, which uses the redudant notation of Nambu spinors, in position basis. We can do this using the Kronecker product of the Pauli matrices with the position basis as follows
\begin{equation}
H =   -\Sigma_j \left[ \mu \sigma_z \ket{j}\bra{j} + (t \sigma_z + i \Delta \sigma_y) \ket{j}\bra{j+1} + (t \sigma_z - i \Delta \sigma_y) \ket{j+1}\bra{j} \right].
\end{equation}


In [2]:
from matplotlib import pyplot as plt
import kwant
import numpy as np
from numpy import linalg as LA
import holoviews as hv
from holoviews import opts
hv.extension('matplotlib', logo=False) #BOKEH DOESNT SUPPORT LATEX IN LABELS
hv.output(fig='svg')

sz = np.array([[1,0],[0,-1]])
sx = np.array([[0,1],[1,0]])
sy = np.array([[0,-1j],[1j,0]])
dims = {'mu':'μ', 'ampli':'$|\psi|^2$', 'x':'$x$', 'E':'$\epsilon$'} #dictionary for dimensions used in plotting

#function that generates 1d Kitaev in Kwant
def kitaev(L=15,m=1,t=1.0, d = None, a=None):
        # On-site Hamiltonian
    if d is None: #if d i not defined, then set equal to hopping parameter
        d = t      
    if a is None:
        a = 1 #lattice parameter
    lat = kwant.lattice.chain(a)
    sys = kwant.Builder()
    sys[(lat(i) for i in range(L))] = -m*sz
            # Hopping
    sys[lat.neighbors()] = -t*sz - d*1j*sy
    return sys
L = 25
chain = kitaev(L).finalized()
ham = chain.hamiltonian_submatrix()
eVal,eVec = LA.eigh(ham)
energies = []
evectors = []
mus = np.arange(0.01,4.1,.2)
for mu in mus:
    sys = kitaev(L=L, m=mu).finalized()
    # Obtain the Hamiltonian as a dense matrix
    ham = sys.hamiltonian_submatrix()
    #diagonalize
    eVal,eVec = LA.eigh(ham)
    energies.append(eVal)
    evectors.append(eVec)
spec=np.asarray(energies)

def wavef(L,mu=0):
    idx = np.where(mus==mu)[0][0]
    zero_mode = np.abs(evectors[idx][:L,L])**2+np.abs(evectors[idx][L:,L])**2 +\
                  np.abs(evectors[idx][:L,L-1])**2+np.abs(evectors[idx][L:,L-1])**2
    first_mode = np.abs(evectors[idx][:L,L+1])**2+np.abs(evectors[idx][L:,L+1])**2 +\
                  np.abs(evectors[idx][:L,L-2])**2+np.abs(evectors[idx][L:,L-2])**2
    wf0 = hv.Path((np.arange(L), zero_mode),kdims=[dims['x'], dims['ampli']]).opts(color='blue', xlim=(0,L-1), ylim=(0,0.5))
    wf1 = hv.Path((np.arange(L),first_mode),kdims=[dims['x'], dims['ampli']]).opts(linestyle='--', color='r',xlim=(0,L-1), ylim=(0,0.5))
    return wf0*wf1

spec_plot = hv.Overlay([hv.Path([(mus,spec[:,i]) for i in range(np.shape(spec)[1])], kdims=[dims['mu'],dims['E']]).opts(color='grey', xlim=(0,4), ylim=(-3,3))])

C:\ProgramData\Anaconda3\envs\lps\lib\site-packages\kwant\solvers\default.py:18: RuntimeWarning: MUMPS is not available, SciPy built-in solver will be used as a fallback. Performance can be very poor in this case.
  "Performance can be very poor in this case.", RuntimeWarning)


In [3]:
spec_plot*hv.HoloMap({mu: hv.VLine(mu).opts(color='blue') for mu in mus},kdims = dims['mu']) +\
hv.HoloMap({mu: wavef(L,mu) for mu in mus}, kdims = dims['mu'])


:Layout
   .HoloMap.I  :HoloMap   [μ]
      :Overlay
         .Path.I  :Path   [μ,$\epsilon$]
         .VLine.I :VLine   [x,y]
   .HoloMap.II :HoloMap   [μ]
      :Overlay
         .Path.I  :Path   [$x$,$|\psi|^2$]
         .Path.II :Path   [$x$,$|\psi|^2$]

The $\mu=0$ is a *pathological* case, where we analytically find that there is one majorana mode $c_1$ localized at one end and another majorana $c_{2L}$ at the opposite end. However, we see that whenever $\mu$ slightly changes, the two modes will 'talk' to each other and the zero-mode is truly the superposition of both the Majorana zero-modes. Now, if we are at $\mu \neq 0$ and go to the thermodynamic limit $L\to\infty$, we induce that the zero-mode also has to be a superposition of these two edge-states, but without any 'talking' happening.